In [3]:
import os
import wave
import numpy as np
import matplotlib.pyplot as plt
from sighelpers import voice_from_signal, format_sig
%matplotlib inline

if __name__ == '__main__':
    good = 0
    for wav_file in os.listdir('train'):#[:40]:
        print(wav_file)
        w = wave.open('train/' + wav_file)
        framerate = w.getframerate()
        frames = w.getnframes()
        channels = w.getnchannels()
        width = w.getsampwidth()       
        data = w.readframes(frames)
        sig = format_sig(data, channels)
        #plt.plot(sig)        
        try:
            tab = voice_from_signal(sig, framerate, low = 15, high = 265, width = 16000, step = 10000)
            avr = np.mean(tab)
            q25 = np.percentile(tab, 25)
            q75 = np.percentile(tab, 75)
            iqr = q75 - q25
            if avr >= 145:
                gender = 'K'
            elif iqr >= 135:
                gender = 'K'
            else:
                gender = 'M'
            print(gender)
            if gender == wav_file[4]:
                good += 1
            else:
                print(avr, iqr)            
        except:
            pass
        print("-----")
    print(good)

057_K.wav


/usr/local/lib/python3.4/dist-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


K
-----
064_M.wav
M
-----
076_M.wav
M
-----
082_M.wav
M
-----
035_M.wav
K
169.651967432 0.0
-----
046_K.wav
K
-----
042_M.wav
M
-----
005_M.wav
M
-----
020_M.wav
M
-----
017_M.wav
M
-----
026_M.wav
M
-----
078_M.wav
M
-----
044_K.wav
-----
030_M.wav


/usr/local/lib/python3.4/dist-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/lib/python3.4/dist-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


M
-----
037_K.wav
K
-----
029_K.wav
K
-----
072_K.wav
K
-----
018_K.wav
K
-----
067_K.wav
K
-----
021_M.wav
M
-----
038_M.wav
M
-----
081_K.wav
K
-----
055_K.wav
K
-----
068_K.wav
K
-----
049_M.wav
M
-----
034_K.wav
K
-----
024_M.wav
M
-----
059_K.wav
-----
073_K.wav
K
-----
065_M.wav
M
-----
071_M.wav
M
-----
032_M.wav
M
-----
033_M.wav
M
-----
001_K.wav
K
-----
058_M.wav
K
212.985990011 0.0
-----
053_M.wav
-----
074_K.wav
K
-----
084_M.wav
M
-----
011_M.wav
M
-----
004_M.wav
M
-----
040_K.wav
K
-----
050_K.wav
K
-----
028_K.wav
-----
010_M.wav
M
-----
075_M.wav
K
152.844316381 28.8488276728
-----
079_K.wav
K
-----
022_K.wav
K
-----
047_K.wav
K
-----
036_K.wav
K
-----
048_K.wav
K
-----
061_M.wav
M
-----
019_M.wav
K
162.032437747 33.603583227
-----
080_M.wav
M
-----
027_M.wav
M
-----
014_K.wav
K
-----
086_K.wav
K
-----
090_M.wav
M
-----
077_K.wav
K
-----
002_M.wav
M
-----
060_K.wav
K
-----
083_K.wav
K
-----
088_K.wav
K
-----
052_M.wav
M
-----
016_K.wav
K
-----
025_K.wav
K
-----
070_M.w